In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import Counter

In [34]:
driver=webdriver.Chrome('e:/chromedriver')
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [35]:
#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a
seoulBtn='#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

In [4]:
driver.find_element_by_css_selector(seoulBtn).click()

In [5]:
allBtn='#mCSB_2_container > ul > li:nth-child(1) > a'

In [6]:
driver.find_element_by_css_selector(allBtn).click()

In [7]:
html=driver.page_source
soup = BeautifulSoup(html, 'html.parser') # 현재 페이지의 소스코드

In [9]:
starbucksSoupList=soup.select('li.quickResultLstCon')

In [10]:
len(starbucksSoupList)
# 스타벅스 서울 매장 수

556

In [25]:
# pr
starbucksSoupList[8].select('i.pin_general')

[]

In [12]:
starbucksList=[]
for item in starbucksSoupList:
    # 매장이름
    name = item.select('strong')[0].text.strip()
    # 위도
    lat = item['data-lat']
    # 경도
    lon = item['data-long']
    # 매장타입
    storeType = item.select('i.pin_general')[0].text
    # split은 문자열만 해당
    # 주소, 전화번호
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    starbucksList.append([name, lat, lon, storeType, address, tel])

In [13]:
starbucksList[555]

['중화역',
 '37.60170912407773',
 '127.07841136432036',
 '리저브 매장 2번',
 '서울특별시 중랑구 봉화산로 35 1,2층',
 '1522-3232']

In [14]:
df = pd.DataFrame(starbucksList, 
                columns=['매장명', '위도','경도', '타입', '주소', '전화번호'])

In [15]:
df.to_excel('seoul_starbucks_list.xlsx', index=False)

In [50]:
sgg_pop_df = pd.read_csv('./data/report.txt', sep='\t', header=2)

In [51]:
sgg_pop_df.head()

,기간,자치구,세대,계,남자,여자,계.1,남자.1,여자.1,계.2,남자.2,여자.2,세대당인구,65세이상고령자
0,2020.3/4,합계,"4,405,833","9,953,009","4,840,912","5,112,097","9,699,232","4,719,170","4,980,062","253,777","121,742","132,035",2.20,"1,552,356"
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"


In [52]:
cols = {
    '기간': 'GIGAN',
    '자치구': 'JACHIGU',
    '계': 'GYE_1',
    '계.1': 'GYE_2',
    '계.2': 'GYE_3',
    '남자': 'NAMJA_1',
    '남자.1': 'NAMJA_2',
    '남자.2': 'NAMJA_3',
    '여자': 'YEOJA_1',
    '여자.1': 'YEOJA_2',
    '여자.2': 'YEOJA_3',
    '세대': 'SEDAE',
    '세대당인구': 'SEDAEDANGINGU',
    '65세이상고령자': 'N_65SEISANGGORYEONGJA'
}

In [53]:
sgg_pop_df.columns

Index(['기간', '자치구', '세대', '계', '남자', '여자', '계.1', '남자.1', '여자.1', '계.2',
       '남자.2', '여자.2', '세대당인구', '65세이상고령자'],
      dtype='object')

In [54]:
sgg_pop_df.rename(columns=cols, inplace=True)
# columns딕셔너리의 키('기간')에 해당되는 이름을 sgg_pop_df 컬럼에서 찾아 cols

In [55]:
sgg_pop_df.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2020.3/4,합계,"4,405,833","9,953,009","4,840,912","5,112,097","9,699,232","4,719,170","4,980,062","253,777","121,742","132,035",2.20,"1,552,356"
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"


In [56]:
sgg_pop_df.info()
# Non-Null Count : 결측치가 아닌 수

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  26 non-null     object 
 1   JACHIGU                26 non-null     object 
 2   SEDAE                  26 non-null     object 
 3   GYE_1                  26 non-null     object 
 4   NAMJA_1                26 non-null     object 
 5   YEOJA_1                26 non-null     object 
 6   GYE_2                  26 non-null     object 
 7   NAMJA_2                26 non-null     object 
 8   YEOJA_2                26 non-null     object 
 9   GYE_3                  26 non-null     object 
 10  NAMJA_3                26 non-null     object 
 11  YEOJA_3                26 non-null     object 
 12  SEDAEDANGINGU          26 non-null     float64
 13  N_65SEISANGGORYEONGJA  26 non-null     object 
dtypes: float64(1), object(13)
memory usage: 3.0+ KB


In [57]:
sgg_pop_df['JACHIGU']

cond = sgg_pop_df['JACHIGU']!='합계'

In [58]:
cond
sgg_pop_df[cond].head(5)

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
1,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2.00,"28,396"
2,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"
3,2020.3/4,용산구,"112,451","244,953","119,074","125,879","229,786","110,604","119,182","15,167","8,470","6,697",2.04,"39,995"
4,2020.3/4,성동구,"136,096","302,695","147,582","155,113","295,591","144,444","151,147","7,104","3,138","3,966",2.17,"45,372"
5,2020.3/4,광진구,"166,857","361,923","174,077","187,846","348,064","168,095","179,969","13,859","5,982","7,877",2.09,"50,047"


In [73]:
sgg_pop_df_selected = sgg_pop_df[sgg_pop_df['JACHIGU']!='합계']

In [60]:
sgg_pop_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  25 non-null     object 
 1   JACHIGU                25 non-null     object 
 2   SEDAE                  25 non-null     object 
 3   GYE_1                  25 non-null     object 
 4   NAMJA_1                25 non-null     object 
 5   YEOJA_1                25 non-null     object 
 6   GYE_2                  25 non-null     object 
 7   NAMJA_2                25 non-null     object 
 8   YEOJA_2                25 non-null     object 
 9   GYE_3                  25 non-null     object 
 10  NAMJA_3                25 non-null     object 
 11  YEOJA_3                25 non-null     object 
 12  SEDAEDANGINGU          25 non-null     float64
 13  N_65SEISANGGORYEONGJA  25 non-null     object 
dtypes: float64(1), object(13)
memory usage: 2.9+ KB


In [74]:
sgg_pop_df_final = sgg_pop_df_selected[['JACHIGU','GYE_1']]

In [75]:
sgg_pop_df_final.columns=['시군구명', '주민등록인구']

In [76]:
sgg_pop_df_final.to_excel('sgg_pop.xlsx', index=False)

In [77]:
sgg_pop_df_final.head()

,시군구명,주민등록인구
1,종로구,"159,842"
2,중구,"135,321"
3,용산구,"244,953"
4,성동구,"302,695"
5,광진구,"361,923"
